# Objective Functions

An **objective function** is a function that creates an evaluation metric for quantifying the performance of a model. 


In a machine learning context, the objective function is interchangeably used with *loss function*, *cost function*, *scoring function*, or *error function*. 

There are general cost functions available to get an idea on how well a model performs, but they are not optimized for problem-specific use cases. With EvalML, we can train and optimize a model for a specific problem by creating and optimizing a problem-specific cost function.

## List of Available Cost Functions
- Fraud Detection
- Lead Scoring

## How to Create a Cost Function
To create a cost function, we need to define three methods: how the cost function should be optimized, how predictions should be made using optimized parameters, and how the performance of a model should be evaluated.